In [1]:
import numpy as np
import tensornetwork as tn
import time
import datetime
from qiskit import QuantumCircuit
from qiskit.quantum_info.operators import Operator
from func_timeout import func_set_timeout
import pandas as pd
import random
import sys


# DEVICE : cpu



In [2]:
def apply_gate(qubit_edges, gate, operating_qubits):
    op = tn.Node(gate)
    for i, bit in enumerate(operating_qubits):
        tn.connect(qubit_edges[bit], op[i])
        qubit_edges[bit] = op[i + len(operating_qubits)]
        
def apply_iostate(qubit_edges,states):
    for k in range(len(qubit_edges)):
        if states[k]==0:
            op = tn.Node(np.array([1,0]))
        else:
            op = tn.Node(np.array([0,1]))
        tn.connect(qubit_edges[k], op[0])

In [3]:
def matrix_2_np(U,q):
    if len(q)!=2:
        return U
    U_new=np.zeros((2, 2, 2, 2), dtype=complex)
    for k1 in range(2):
        for k2 in range(2):
            for k3 in range(2):
                for k4 in range(2):
                    U_new[k1][k2][k3][k4]=U[2*k2+k1][2*k4+k3]
    return U_new

def np_2_matrix(U,q_num):
    U_new=np.zeros((2**q_num,2**q_num), dtype=complex)
    for k1 in range(2**q_num):
        for k2 in range(2**q_num):
            b1=[int(b1) for b1 in list(bin(k1)[2:])]
            for k3 in range(q_num-len(b1)):
                b1.insert(0,0)
            b2=[int(b2) for b2 in list(bin(k2)[2:])]
            for k3 in range(q_num-len(b2)):
                b2.insert(0,0)            
            temp=U
            for b in b1:
                temp=temp[b]
            for b in b2:
                temp=temp[b]
            U_new[k1][k2]=temp
    return U_new

In [4]:
def get_real_qubits_num(cir):
    gates=cir.data
    q=0
    for k in range(len(gates)):
        q=max(q,max([qbit.index for qbit in gates[k][1]]))
    return q+1

In [5]:
@func_set_timeout(3600)
def Simulation_with_TensorNetwork(cir,io=False):
    all_nodes = []
    with tn.NodeCollection(all_nodes):
        qubits_num=get_real_qubits_num(cir)
        state_nodes = [
            tn.Node(np.eye(2, dtype=complex)) for _ in range(qubits_num)
        ]
        
        qubits0=[node[0] for node in state_nodes]
        qubits = [node[1] for node in state_nodes]
        gates=cir.data
        for k in range(len(gates)):
            U=Operator(gates[k][0]).data
            q=[qbit.index for qbit in gates[k][1]]
            if len(q)==2:
                U=matrix_2_np(U,q)
            apply_gate(qubits, U, q)
    
    
        if io:
            input_state=[random.randint(0,1) for k in range(qubits_num)]
            output_state=[random.randint(0,1) for k in range(qubits_num)]
            
            apply_iostate(qubits0,input_state)
            apply_iostate(qubits,output_state)
            qubits0=[]
            qubits=[]
        edge_order=qubits0+qubits
#         print(edge_order)
        result = tn.contractors.auto(all_nodes, output_edge_order=edge_order).tensor
        return result

In [6]:
def Simulation_with_time_out(cir,io=False):
    try:
        return Simulation_with_TensorNetwork(cir,io)
    except:
        return 0

In [13]:
tn.set_default_backend('tdd')
path="Benchmarks/"
file_name="qft_11.qasm"
# path='7x7_cz/'
# file_name="inst_7x7_20_0.qasm"
cir=QuantumCircuit.from_qasm_file(path+file_name)
t_start= time.time()
U=Simulation_with_TensorNetwork(cir)
t_end=time.time()
print('Time:',t_end-t_start)
# qubits_num=get_real_qubits_num(cir)
# U=np_2_matrix(U,qubits_num)
# print(U[0][0])

C:\Users\YINGTE~1\AppData\Local\Temp/ipykernel_17032/2942288214.py:5: DeprecationWarning: Back-references to from Bit instances to their containing Registers have been deprecated. Instead, inspect Registers to find their contained Bits.
  q=max(q,max([qbit.index for qbit in gates[k][1]]))
C:\Users\YINGTE~1\AppData\Local\Temp/ipykernel_17032/156966690.py:15: DeprecationWarning: Back-references to from Bit instances to their containing Registers have been deprecated. Instead, inspect Registers to find their contained Bits.
  q=[qbit.index for qbit in gates[k][1]]


In [17]:
# print(U.dtype)
# print(U.shape)
# print(U.size)
# print(U.itemsize)
# print(U.nbytes)
# print(sys.getsizeof(U))
# print(type(U))
# f=open('data_u.txt','wb')
# np.save(f, U)
# f.close()

In [ ]:
# # path='bv/'
# path="Gen_QFT/"
# # file_list = open('test3.txt','r')
# # excel_path = 'data2.xlsx'
# # path="Gen_SWAP/"
# file_list = open('test4.txt','r')
# excel_path = 'data1225.xlsx'
# # result=[]
# # with open('computed.txt','r') as f:
# #     for line in f:
# #         result.append(line[:-1])

# data = ["New"]
# data=np.array(data).reshape(1,len(data))
# df = pd.read_excel(excel_path, header=None)
# ds = pd.DataFrame(data)
# df = df.append(ds, ignore_index=True)
# df.to_excel(excel_path, index=False, header=False)

# for file in file_list.readlines():
#     file_name=file[:-1]
#     print(file_name)
    
#     cir=QuantumCircuit.from_qasm_file(path+file_name)
#     qubits_num=get_real_qubits_num(cir)
    
#     data = []
#     data.append(file_name)
#     data.append(qubits_num)
    
#     time_now = datetime.datetime.now()
# #     print(time_now.strftime('%m.%d-%H:%M:%S'))
    
#     t_start = time.time()
#     res=Simulation_with_time_out(cir,True)
#     run_time=time.time()-t_start
#     print(run_time)
#     data.append(run_time)
#     if isinstance(res,int):
#         data.append(-1)
        
    
#     data=np.array(data).reshape(1,len(data))
#     df = pd.read_excel(excel_path, header=None)
#     ds = pd.DataFrame(data)
#     df = df.append(ds, ignore_index=True)
#     df.to_excel(excel_path, index=False, header=False)
#     if isinstance(res,int):
#         break
# file_list.close()

qft_20.qasm
0.21600031852722168
qft_21.qasm
1.0810060501098633
qft_22.qasm
0.5509994029998779
qft_23.qasm
0.5129718780517578
qft_24.qasm
0.3509998321533203
qft_25.qasm
0.4380021095275879
qft_26.qasm
0.6779987812042236
qft_27.qasm
1.0389511585235596
qft_28.qasm
1.1199791431427002
qft_29.qasm
6.929980754852295
qft_30.qasm
3.956935167312622
qft_31.qasm
